<a href="https://colab.research.google.com/github/IGVF/seqspec/blob/main/examples/seqspec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --quiet https://github.com/igvf/seqspec

In [2]:
!cd seqspec && pip --quiet install .

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [3]:
!seqspec

usage: seqspec [-h] <CMD> ...

seqspec 0.0.0: Format sequence specification files

positional arguments:
  <CMD>
    check     validate seqspec file
    format    format seqspec file
    print     print seqspec file

optional arguments:
  -h, --help  show this help message and exit


In [4]:
!head seqspec/assays/10x-RNA-v2/spec.yaml

!Assay
name: 10x-RNA-v2
doi: https://doi.org/10.1038/ncomms14049
description: The 10x Genomics Chromium Single Cell 3' Solution V2 chemistry
modalities:
- RNA
lib_struct: https://teichlab.github.io/scg_lib_structs/methods_html/10xChromium3.html
assay_spec:
  - !Region
    region_id: RNA


In [5]:
!seqspec print /content/seqspec/assays/10x-RNA-v2/spec.yaml

# 10x-RNA-v2
- DOI: [https://doi.org/10.1038/ncomms14049](https://doi.org/10.1038/ncomms14049)
- Description: The 10x Genomics Chromium Single Cell 3' Solution V2 chemistry
- Modalities: RNA
    
## Final Library
###### RNA
<pre style="overflow-x: auto; text-align: left; background-color: #f6f8fa">AATGATACGGCGACCACCGAGATCTACACTCTTTCCCTACACGACGCTCTTCCGATCTNNNNNNNNNNNNNNNNNNNNNNNNNNXXAGATCGGAAGAGCACACGTCTGAACTCCAGTCACNNNNNNNNATCTCGTATGCCGTCTTCTGCTTG</pre>
1. <details><summary>Illumina P5</summary>

   - sequence_type: fixed
   - sequence: <pre style="overflow-x: auto; text-align: left; margin: 0; display: inline;">AATGATACGGCGACCACCGAGATCTACAC</pre>
   - min_len: 29
   - max_len: 29
   - onlist: None
   </details>
2. <details><summary>Truseq Read 1</summary>

   - sequence_type: fixed
   - sequence: <pre style="overflow-x: auto; text-align: left; margin: 0; display: inline;">TCTTTCCCTACACGACGCTCTTCCGATCT</pre>
   - min_len: 10
   - max_len: 10
   - onlist: None
   </details>
3. <details>

In [6]:
!seqspec check /content/seqspec/assays/10x-RNA-v2/spec.yaml

[error 1] None is not of type 'string' in spec['assay_spec'][0]['join']['regions'][2]['onlist']['md5']
[error 2] None is not of type 'string' in spec['assay_spec'][0]['join']['regions'][7]['onlist']['md5']
